In [1]:
import pandas as pd
train=pd.read_csv('train.csv',index_col=0)
test=pd.read_csv('test.csv',index_col=0)

#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)

import catboost as cb
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

model = cb.CatBoostClassifier(iterations=1000, 
                              depth=7, 
                              learning_rate=0.01, 
                              loss_function='Logloss', 
                              eval_metric='AUC',
                              logging_level='Verbose', 
                              metric_period=50
                             )

# 得到分类特征的列号
categorical_features_indices = []
for i in range(len(X_train.columns)):
    if X_train.columns.values[i] in attr:
        categorical_features_indices.append(i)
print(categorical_features_indices)

model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=categorical_features_indices)

#model = cb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test)
#predict = model.predict_proba(test)
#print(predict)
test['Attrition']=predict
## 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)

[0, 1, 3, 5, 6, 9, 13, 15, 19, 20]
0:	test: 0.6390374	best: 0.6390374 (0)	total: 80ms	remaining: 1m 19s
50:	test: 0.7893703	best: 0.7893703 (50)	total: 957ms	remaining: 17.8s
100:	test: 0.7950453	best: 0.7950453 (100)	total: 2s	remaining: 17.8s
150:	test: 0.8011568	best: 0.8011568 (150)	total: 3.09s	remaining: 17.4s
200:	test: 0.7958092	best: 0.8011568 (150)	total: 4.43s	remaining: 17.6s
250:	test: 0.8023573	best: 0.8023573 (250)	total: 5.64s	remaining: 16.8s
300:	test: 0.8034487	best: 0.8034487 (300)	total: 6.66s	remaining: 15.5s
350:	test: 0.8046491	best: 0.8046491 (350)	total: 8.02s	remaining: 14.8s
400:	test: 0.8030121	best: 0.8046491 (350)	total: 9.41s	remaining: 14.1s
450:	test: 0.8054131	best: 0.8054131 (450)	total: 10.8s	remaining: 13.1s
500:	test: 0.8067227	best: 0.8067227 (500)	total: 12.1s	remaining: 12.1s
550:	test: 0.8062862	best: 0.8067227 (500)	total: 13.3s	remaining: 10.8s
600:	test: 0.8056313	best: 0.8067227 (500)	total: 14.5s	remaining: 9.64s
650:	test: 0.8053039	best

In [3]:
test.reset_index()[['user_id','Attrition']].to_csv('submit_cb.csv')